In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dash import Dash
from pymongo import MongoClient
# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME: Update with your username and password and CRUD Python module name
username = "aacuser"
password = "Cixin7945$"

# Connect to the database via CRUD Module
db = AnimalShelter(username, password)

# Class read method must support the return of a list object and accept projection JSON input
# Sending the read method an empty document requests all documents to be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column, and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove it in the dataframe here.
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME: Add Grazioso Salvare’s logo
image_filename = '/home/meharbansingh_snhu/Desktop/Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# FIX ME: Place the HTML image tag in the line below into the app.layout code according to your design
# FIX ME: Also remember to include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard Meharban Singh'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'max-width':'30%','max-height':'30%'}),
    html.Hr(),
 
    html.Div(
        className='buttonRow',
        style={'display': 'flex'},
        children=[
            dcc.RadioItems(
                id='rescue-type-radio',
                options=[
                    {'label': 'Water', 'value': 'Water'},
                    {'label': 'Mountain or Wilderness', 'value': 'Mountain or Wilderness'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
                ],
               
                labelStyle={'display': 'inline-block'}
            ),

            html.Button('Reset', id='reset-button', n_clicks=0, style={'margin-left': '20px'})
        ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        # If you completed the Module Six Assignment, you can copy in the code you created here
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
       
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
   
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update the update_dashboard callback function to include the radio button values
@app.callback(Output('datatable-id', 'data'),
              [Input('rescue-type-radio', 'value'),
               Input('reset-button', 'n_clicks')])
def update_dashboard(rescue_type, reset_clicks):
    #setup reset button
    if reset_clicks is not None and reset_clicks>0:
        rescue_type = None
    #do query according to button press
    if rescue_type == 'Water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'Mountain or Wilderness':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'Disaster or Individual Tracking':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query={} #default

    data = pd.DataFrame.from_records(db.read(query))
    data.drop(columns=['_id'], inplace=True)
   
    return data.to_dict("records")


# Display the breeds of animal based on quantity represented in the data table
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return

    return dcc.Graph(figure=px.pie(dff, names='breed', title='Preferred Animals'))


# This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'), [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns, **kwargs):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geolocation chart for the selected data entry
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return
    row = index[0] if index else 0

    # Austin TX is at [30.75,-97.48]
    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97.48],
        zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                children=[
                    dl.Tooltip(dff.iloc[row, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9])
                    ])
                ]
            )
        ]
    )


app.run_server(debug=True)



Dash is running on http://127.0.0.1:12552/

Dash app running on http://127.0.0.1:12552/
